In [1]:
import torch.nn as nn
import cv2
from torchvision import transforms
import torch
import torchvision
import numpy as np
import ds_ear
import glob
from PIL import Image
from matplotlib import image
from os.path import join


CATEGORIES = ["Falco", "Jesse", "Johannes", "Konrad", "Nils"]
AUTHORIZED = ["Falco","Konrad"]
RESIZE_Y = 150
RESIZE_X = 100
DATA_TEST_FOLDER = "../test/*png"


def get_data(folder):
    img_array = []
    img_array_resized = []
    files = glob.glob (folder)
    for idx, f in zip(range(len(files)),files):
        image = cv2.imread(f)
        img_array.append (image)
        img_array_resized.append(cv2.resize(img_array[idx],(RESIZE_Y,RESIZE_X)))
    return np.asarray(img_array_resized)


model = torch.load('./class_sample/model.pt')



In [2]:
image_array = []
files = glob.glob (DATA_TEST_FOLDER)
files.sort()
for idx, f in zip(range(len(files)),files):
    image = Image.open(f)
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Resize((RESIZE_Y, RESIZE_X)),
        torchvision.transforms.Lambda(lambda x: x.convert('RGB')),
        torchvision.transforms.ToTensor(),

        torchvision.transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225]
        )
    ])
    image_transformed = transform(image)
    image_transformed = image_transformed.reshape(-1, RESIZE_Y, RESIZE_X, 1)
    image_transformed = image_transformed.permute(3, 0, 1, 2)
    image_array.append(image_transformed.type('torch.FloatTensor'))
    #image_array.append(image_transformed.type('torch.cuda.FloatTensor'))


In [3]:
all_classes = []
for i in image_array:
	with torch.no_grad():
		pred = model(i)
		pred = torch.softmax(pred, 1)
		pred = pred.cpu().numpy()

	classes = np.argmax(pred, 1)
	all_classes.append(classes[0])

	pred = np.append(pred, classes)
	pred = np.append(pred, CATEGORIES[classes[0]])	
	print(pred, "\n")
print(all_classes)


['0.789467990398407' '0.031321123242378235' '0.035172224044799805'
 '0.038966841995716095' '0.10507182776927948' '0.0' 'Falco'] 

['0.6517340540885925' '0.04698029160499573' '0.06975772231817245'
 '0.07839997857809067' '0.15312793850898743' '0.0' 'Falco'] 

['0.7030173540115356' '0.25509652495384216' '0.0046354029327631'
 '0.010346267372369766' '0.02690448798239231' '0.0' 'Falco'] 

['0.8828981518745422' '0.02076706290245056' '0.008723532781004906'
 '0.04275737330317497' '0.04485388845205307' '0.0' 'Falco'] 

['0.9697948098182678' '0.021453814581036568' '0.00019798381254076958'
 '0.0008644312620162964' '0.0076888869516551495' '0.0' 'Falco'] 

['0.8816684484481812' '0.07324919104576111' '0.0010174292838200927'
 '0.003983926959335804' '0.040081024169921875' '0.0' 'Falco'] 

['0.8315247297286987' '0.04502866417169571' '0.0008458319352939725'
 '0.00120946962852031' '0.12139128148555756' '0.0' 'Falco'] 

['0.8306043148040771' '0.07410381734371185' '0.0014230861561372876'
 '0.013272075913846

In [4]:
# Hier besser die Warscheinlichkeit über alle Bilder ermitteln und darüber prüfen.
# Beispiel: Bei 5 Bilder muss die aufsummierte Wahrscheinlichkeit für eine Person >4 sein!!

NUMBER_AUTHORIZED = int(.8*len(image_array))
authentification_dict = {CATEGORIES[i]:all_classes.count(i) for i in all_classes}
print(authentification_dict) 

for a in authentification_dict:
    if a in AUTHORIZED and authentification_dict[a] >= NUMBER_AUTHORIZED:
        print("Acces granted! Welcome "  + a + "!")
    else:
        print("Acces denied")
        break


{'Falco': 9, 'Konrad': 1}
Acces granted! Welcome Falco!
Acces denied
